# Initial model for EVs - version 1

11/01/2024

Key assumptions:
- Time of arrival and departure, Ta and Td respectively, of each EV is known (this can be randomly generated for further impolementation)
- Every EV has possibility to connect to a charger

Objectives:

$ min ( P^{max}_t) $  for all t, where   $ P^{max}_t = max( sum( P_{s,t})) $ over time t

$ max( P_{s,t} \cdot u_{s,t}) $ for all s, for all t

S. t. :

$ 0 <= sum(P_{s,t}) <= P^{max}_t $ for all t 

$ 0 <= P_{s,t} <= P^{max}_{s,t} \cdot u_{s,t} $ for all s, for all t

$ SOC^{+}_{s,t} >= SOC^{min} $ for all s, for all t


In [149]:
import pyomo.environ as pyo
import numpy as np
import pandas as pd

# Generate the model: 
model = pyo.ConcreteModel()

# import data from Excel file
def readInputFile(filename):
    EV_data = pd.read_excel(filename, sheet_name= 'EVs', index_col=0)
    return {'EV_data':EV_data}

filename = 'EV_model.xlsx'
data = readInputFile(filename)
# ----------------------------
EV_data = data['EV_data']



# Define sets
model.T = pyo.RangeSet(1,24)                                        # time periods
model.S = pyo.Set(initialize=EV_data.index, ordered=True)           # EVs

# Define parameters
model.Pmax = pyo.Param(model.S, initialize=EV_data['Pmax_kw'].to_dict())        # Maximum charging power
model.Ta = pyo.Param(model.S, initialize= EV_data['Ta'].to_dict())              # Arrival time
model.Td = pyo.Param(model.S, initialize= EV_data['Td'].to_dict())              # Departure time
model.u = pyo.Param(model.T, model.S, mutable=True, domain=pyo.Binary)          # The s-th EV is connected (binary)
model.remaining_time = pyo.Param(model.T, model.S, mutable=True)                # Hours of connection remaining

#def initialize_parameters(model):
for s in model.S:
    u = np.zeros(len(model.T))
    remaining_time = np.zeros(len(model.T))
    u[model.Ta[s]:model.Td[s]] = 1
    tot = sum(u)
    remaining_time[model.Ta[s]:model.Td[s]] = [-e + sum(u) for e in range(model.Td[s] - model.Ta[s])]
    for t in model.T:
        model.u[t,s] = u[t-1]
        model.remaining_time[t,s] = remaining_time[t-1]


NOTE: In the parameter U the simulation will happen. Meaning that when forecasting/random numbers and scheduling will be implemented, all the research will be given on these values. Sensitivity analysis will be also computed in model.u. When changing this matrix, the solutions will change as well.

In [150]:

# Define variables
model.P = pyo.Var(model.T, model.S, domain=pyo.NonNegativeReals)     # Charging power





# Optimization problem for each hour:

def sub_opt(model, t):

    #defining the objective function:
    model.OF1 = pyo.Objective(rule=(sum(model.P[t,s]) for s in model.S), sense=pyo.minimize)





# Defining objective functions:


# Optimization model for each hour



#for t in model.T:
#    model.OF.add(expr=max(sum(model.P[t, s]) for s in model.S), sense=pyo.minimize)
    #for s in model.S:
    #    model.OF.add(expr=(model.P[t,s]*model.u[t,s]), sense=pyo.maximize)
